In [2]:
def vector_addition(*vectors):
    '''Extends all vectors with zero so they have the shape of the biggest one, then add them'''
    if not vectors:
        return None
    size = max(len(v) for v in vectors)
    for v in vectors :
        v += [0]*(size-len(v))
    return [sum(v) for v in zip(*vectors)]

def extend_vector(v, n):
    return [0]*n+v

class Objective_Analyzer():
    def __init__(self):
        self.conclusions=[]
        self.parameters=dict()
    def compute_vector(self, sentence):
        return vector_addition(*[self.parameters[word] for word in sentence if word in self.parameters])
    def take_decision(self, vector):
        #TO BE CHANGED
        if not vector:
            return None
        return self.conclusions[vector.index(max(vector))]
    def analyze(self, sentence_in_string):
        return self.take_decision(self.compute_vector(self.prepare_sentence(sentence_in_string)))
    def prepare_sentence(self, sentence_in_string):
        return sentence_in_string.split()
    def add_words_and_weights(self, words, weight_vectors, rank=0):
        for word, vector in zip(words, weight_vectors):
            assert len(vector)+rank <= len(self.conclusions), 'Too many weights for {} : {}'.format(word, vector)
            if word in self.parameters :
                self.parameters[word]=vector_addition(self.parameters[word], extend_vector(vector, rank))
            else :
                self.parameters[word]=extend_vector(vector, rank)
    def add_to_guess(self, guess_conclusions, words_weights=None): #TO COMPLETE
        #self.add_words_and_weights()#....
        self.conclusions+=guess_conclusions
        

In [3]:
recurrence=Objective_Analyzer()

In [4]:
recurrence.add_to_guess(['reccurent', 'not recurrent'])

In [5]:
recurrence.add_words_and_weights(['every','once'],[[1,0],[0,1]])

In [6]:
recurrence.add_words_and_weights(['frequent','intermittent', 'periodic',
                                  'recurring', 'repeated', 'repetitive', 'alternate', 'chain',
                                  'continued', 'cyclical', 'habitual', 'isochronal', 'isochronous',
                                  'periodical', 'regular', 'reoccurring', 'rolling'],
                                 [[1,0] for _ in range(6)]+[[0.5,0] for _ in range(11)])

In [7]:
print(recurrence.analyze('actualize API once'))

not recurrent
